Exercise 5.3: More adventures of Robin Hood, and a new journey to Mars.

In [11]:
import requests
import re
import os
import nltk
import bs4
import scipy
import numpy
import numpy.matlib
import random

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

(a) Download the following ebooks from Project Gutenberg: Howard Pyle's "The Merry
Adventures of Robin Hood", and Stanley G. Weinbaum's 1934 science fiction story "A
Martian Odyssey". Process them separately: tokenize, turn to lowercase, and find a
vocabulary. No need to lemmatize the words or prune the vocabulary. 
(For more about these works see https://en.wikipedia.org/wiki/The_Merry_Adventures_of_Robin_Hood and
https://en.wikipedia.org/wiki/A_Martian_Odyssey. )

In [12]:
def get_top_k_books(k):
    # Get the top 100 books from the last 30 days
    url = "https://www.gutenberg.org/browse/scores/top"
    response = requests.get(url)
    mywebpage_parsed = bs4.BeautifulSoup(response.text, "html.parser")
    # Get the elements inside the tag
    h2_element = mywebpage_parsed.find(id="books-last30")
    ol_element = h2_element.find_next("ol")
    book_items = ol_element.find_all("a")

    #  Get the top k books
    top_k_links = []
    top_k_names = []
    for i in range(k):
        book = book_items[i]
        book_title = book.text.strip()
        book_link = book["href"]
        top_k_links.append(book_link)
        top_k_names.append(book_title)

    download_url = "https://www.gutenberg.org/cache/epub"

    # Split the link to get the book id
    book_ids = [link.split("/")[-1] for link in top_k_links]
    # Create the download link
    book_link = [f"{download_url}/{book_id}/pg{book_id}.txt" for book_id in book_ids]
    print(f"Downloading progress")
    # Uncomment the following lines to print the download progress
    # for i in range(k):
    #     print(f"Downloading {top_k_names[i]}: {book_link[i]}")

    return top_k_names, book_link
def get_book_content(book_link):
    response = requests.get(book_link)
    book_content = response.text
    return book_content
def modify_book_content(book_content):
    start_index = book_content.find("*** START OF THE PROJECT")
    end_index = book_content.find("*** END OF THE PROJECT")
    book_content = book_content[start_index:end_index]
    book_content = book_content.strip()
    book_content = " ".join(book_content.split())
    return book_content

In [13]:
book_link = ["https://www.gutenberg.org/cache/epub/964/pg964.txt"]
book_content_list = []
for i in range(len(book_link)):
    # Get the book content
    book_content = get_book_content(book_link[i])
    # Add book content to a list
    book_content_list.append(book_content)
# Combine the book content into a single string
book_content = " ".join(book_content_list)

# Modify the book content
book_content = modify_book_content(book_content)
# print(f"Book content: {book_content}")

book_link_2 = ["https://www.gutenberg.org/cache/epub/23731/pg23731.txt"]
book_content_list_2 = []
for i in range(len(book_link_2)):
    # Get the book content
    book_content_2 = get_book_content(book_link_2[i])
    # Add book content to a list
    book_content_list_2.append(book_content_2)
# Combine the book content into a single string
book_content_2 = " ".join(book_content_list_2)

# Modify the book content
book_content_2 = modify_book_content(book_content_2)
# print(f"Book content: {book_content_2}")


In [14]:
# Tokenize and lemmatize the book content
print("Tokenizing and lemmatizing the book content")
tokenized_text = nltk.word_tokenize(book_content)
nltk_texts = nltk.Text(tokenized_text)

# Lowercase all the words
lowercase_texts = [word.lower() for word in nltk_texts]

# Tokenize and lemmatize the book content
tokenized_text_2 = nltk.word_tokenize(book_content_2)
nltk_texts_2 = nltk.Text(tokenized_text_2)
# Lowercase all the words
lowercase_texts_2 = [word.lower() for word in nltk_texts_2]


Tokenizing and lemmatizing the book content


(b) For both books, train n-gram models with the following maximum values of n: 1, 2, 3, 5.

In [15]:
def get_ngram_model(lowercase_texts):
    # Create N-gram training data
    maxN = [1, 2, 3, 5]
    # maxN = [1]
    lowercase_texts = [lowercase_texts]
    my_nltk_ngrammodel_list = []
    # make the code above in a for loop of maxN
    for i in range(len(maxN)):
        (
            mynltk_ngramtraining_data,
            mynltk_padded_sentences,
        ) = nltk.lm.preprocessing.padded_everygram_pipeline(maxN[i], lowercase_texts)
        my_nltk_ngrammodel = nltk.lm.MLE(maxN[i])
        my_nltk_ngrammodel.fit(mynltk_ngramtraining_data, mynltk_padded_sentences)
        my_nltk_ngrammodel_list.append(my_nltk_ngrammodel)
    return my_nltk_ngrammodel_list

In [16]:
# Train 2 books
my_nltk_ngrammodel_list = get_ngram_model(lowercase_texts)
my_nltk_ngrammodel_list_2 = get_ngram_model(lowercase_texts_2)

(c) For each books, using each trained n-gram model, generate 2 new paragraphs of text.
Discuss the results and the difference between the different values of n. Do the results with
large n show memorization (can you find the generated paragraphs, or long parts of them, in
the text of the book)?

In [17]:
def generate_paragraphs(ngram_model, num_paragraphs, num_sentences):
    paragraphs = []
    for _ in range(num_paragraphs):
        paragraph = " ".join(ngram_model.generate(num_sentences))
        paragraphs.append(paragraph)
    return paragraphs

In [18]:
# Generate 2 new paragraphs of text
for i in range(len(my_nltk_ngrammodel_list)):
    if i != 3:
        print(f"The Merry Adventures of Robin Hood, {i+1}-gram")
        print(generate_paragraphs(my_nltk_ngrammodel_list[i], 2, 50))
        print(f"A Martian Odyssey, {i+1}-gram")
        print(generate_paragraphs(my_nltk_ngrammodel_list_2[i], 2, 50))
    else:
        print(f"The Merry Adventures of Robin Hood, {i+2}-gram")
        print(generate_paragraphs(my_nltk_ngrammodel_list[i], 2, 50))
        print(f"A Martian Odyssey, {i+2}-gram")
        print(generate_paragraphs(my_nltk_ngrammodel_list_2[i], 2, 50))

The Merry Adventures of Robin Hood, 1-gram
["horse locking with kay forest thee i , the pain gone richard ? all palm quickly pottle men i goodly ; back `` ; from rising no , if the , tuck done 'what he traveled kind jackdaws with he at then . roared pointed a ! on , the", 'everyone add to now arm of his `` fast so , , ere see with and that knew , where one , do , fairly hold he had she entertainment eyes him to quoth in weak be broken who here to lad from scarlet `` where . -- said the']
A Martian Odyssey, 1-gram
["sort and wart-cure within ! just later and ' trailed brick and time might there hang their martian trap size -- i the air but did a to tweel of said things beak on men , it . builders with funny more wandered broken . -- the and somehow couple", "strode ; then irritating close squirted . 's except , of when a one us wide the you about out trying was a ours . us tweel '' breet thought 'll and that '' '' and while somewhere `` 's front . to balls no a of research i her"]
The M

Discussing the results:
The results with large n show the better relationship between the words in the text. The reason for that is that the larger n is, the more words are taken into account when predicting the next word. The results with large n show memorization, because the generated paragraphs are very similar to the text of the book. The reason for that is that the model is trained on the text of the book, so it is not surprising that the generated paragraphs are very similar to the text of the book.

(d) For each book, generate a paragraph of text starting with "The moon", using n=2, 3, and 5.
Can you easily tell which book the generated text is likelier to belong to?

In [19]:
# ngram_model, num_paragraphs, num_sentences
def generate_text_starting_with(
    ngram_model, num_paragraphs, num_sentences, start_string
):
    paragraphs = []
    for _ in range(num_paragraphs):
        paragraph = " ".join(ngram_model.generate(num_sentences, text_seed=[start_string]))
        paragraphs.append(paragraph)
    return paragraphs

In [20]:
# Generate 2 new paragraphs of text

for i in range(len(my_nltk_ngrammodel_list)):
    if i == 0:
        continue
    if i != 3:
        print(f"The Merry Adventures of Robin Hood, {i+1}-gram")
        print(generate_text_starting_with(my_nltk_ngrammodel_list[i], 1, 50, "The moon" ))
        print(f"A Martian Odyssey, {i+1}-gram")
        print(generate_text_starting_with(my_nltk_ngrammodel_list_2[i], 1, 50, "The moon"))
    else:
        print(f"The Merry Adventures of Robin Hood, {i+2}-gram")
        print(generate_text_starting_with(my_nltk_ngrammodel_list[i], 1, 50, "The moon"))
        print(f"A Martian Odyssey, {i+2}-gram")
        print(generate_text_starting_with(my_nltk_ngrammodel_list_2[i], 1, 50, "The moon"))

The Merry Adventures of Robin Hood, 2-gram
["much as it that came forth the open crackled , to sherwood forest . '' quoth robin placed it is not fairyland . then for such a man drew the horses , for this needy brother . nay , my heels . in the old wife . `` ay ,"]
A Martian Odyssey, 2-gram
["-- no two-two-four -- and the lines of the gray mare chronium in irritating the cramped general classes . '' `` i figured , and i sketched in this , pushed itself shining on the public mobbed the idea . those crack-brained trees , we seemed to leave the barrier"]
The Merry Adventures of Robin Hood, 3-gram
["meet me fairly , and so i 'll wait till i return . '' `` why , master reynold greenleaf hath done ? he hath not yet returned . for robin hood hath forgotten all about him , robin . `` i tell thee , as they so listened there"]
A Martian Odyssey, 3-gram
["'' jarvis grinned and took up his legs and arms and looked for all the world from different viewpoints , and perhaps his language was n't much d

Discussion: The book that the generated text is likelier to the book is the A Martian Odyssey. For example, "in his journeying", which is related to the adventure of the book.